<a href="https://colab.research.google.com/github/rooroogjoob/cis700_hw2/blob/master/my_game_improved.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text adventure game

This Python notebook builds a simple text advenutre game inspired by the [Adventuron Classroom](https://adventuron.io/classroom/) design by Chris Ainsley of Adventuron Software Limited.

The main components are:
1. __The parser__, which interprets the player's commands.
2. __The game__, which represents the world (a collection of __locations__ and __items__), and describes what the player sees.
3. __The data__, which you input to create your own unique game.

## The Game Class
The game keeps track of the state of the world, and describes what the player sees as they move through different locations.

In [0]:
class Game:
  """The Game class represents the world.  Internally, we use a 
     graph of Location objects and Item objects, which can be at a 
     Location or in the player's inventory.  Each locations has a set of
     exits which are the directions that a player can move to get to an
     adjacent location. The player can move from one location to another
     location by typing a command like "Go North".
  """

  def __init__(self, start_at):
    # start_at is the location in the game where the player starts
    self.curr_location = start_at
    self.curr_location.has_been_visited = True
    # inventory is the set of objects that the player has collected/
    self.inventory = {}
    # Print the special commands associated with items in the game (helpful 
    # for debugging and for novice players).
    self.print_commands = True

  def describe(self):
    """Describe the current game state by first describing the current 
       location, then listing any exits, and then describing any objects
       in the current location."""
    self.describe_current_location()
    self.describe_exits()
    self.describe_items()

  def describe_current_location(self):
    """Describe the current location by printing its description field."""
    print(self.curr_location.description)

  def describe_exits(self):
    """List the directions that the player can take to exit from the current
       location."""
    exits = []
    for exit in self.curr_location.connections.keys():
      exits.append(exit.capitalize())
    if len(exits) > 0:
      print("Exits: ", end = '')
      print(*exits, sep = ", ",)
  
  def describe_items(self):
    """Describe what objects are in the current location."""
    if len(self.curr_location.items) > 0:
      print("You see: ")
      for item_name in self.curr_location.items:
        item = self.curr_location.items[item_name]
        print(item.description)
        if self.print_commands:
          special_commands = item.get_commands()
          for cmd in special_commands:
            print('\t', cmd)

  def add_to_inventory(self, item):
    """Add an item to the player's inventory."""
    self.inventory[item.name] = item
  
  def is_in_inventory(self,item):
    return item.name in self.inventory

  def get_items_in_scope(self):
    """Returns a list of items in the current location and in the inventory"""
    items_in_scope = []
    for item_name in self.curr_location.items:
      items_in_scope.append(self.curr_location.items[item_name])
    for item_name in self.inventory:
      items_in_scope.append(self.inventory[item_name])
    return items_in_scope

## Locations

Locations Locations are the places in the game that a player can visit.  They contain connects to other locations and items that the player can interact with.

In [0]:
class Location:
  """Locations are the places in the game that a player can visit.
     Internally they are represented nodes in a graph.  Each location stores
     a description of the location, any items in the location, its connections
     to adjacent locations, and any blocks that prevent movement to an adjacent
     location.  The connections is a dictionary whose keys are directions and
     whose values are the location that is the result of traveling in that 
     direction.  The travel_descriptions also has directions as keys, and its 
     values are an optional short desciption of traveling to that location.
  """
  def __init__(self, name, description, end_game=False):
    # A short name for the location
    self.name = name
    # A description of the location
    self.description = description
    # True if entering this location should end the game
    self.end_game = end_game
    # Dictionary mapping from directions to other Location objects
    self.connections = {}
    # Dictionary mapping from directions to text description of the path there
    self.travel_descriptions = {}
    # Dictionary mapping from item name to Item objects present in this location
    self.items = {}
    # Dictionary mapping from direction to Block object in that direction
    self.blocks = {}
    # Flag that gets set to True once this location has been visited by player
    self.has_been_visited = False

  def add_connection(self, direction, connected_location, travel_description=""):
    """Add a connection from the current location to a connected location.
       Direction is a string that the player can use to get to the connected
       location.  If the direction is a cardinal direction, then we also 
       automatically make a connection in the reverse direction."""
    self.connections[direction] = connected_location
    self.travel_descriptions[direction] = travel_description
    if direction == 'north':
      connected_location.connections["south"] = self
      connected_location.travel_descriptions["south"] = ""
    if direction == 'south':
      connected_location.connections["north"] = self
      connected_location.travel_descriptions["north"] = ""
    if direction == 'east':
      connected_location.connections["west"] = self
      connected_location.travel_descriptions["west"] = ""
    if direction == 'west':
      connected_location.connections["east"] = self
      connected_location.travel_descriptions["east"] = ""
    if direction == 'up':
      connected_location.connections["down"] = self
      connected_location.travel_descriptions["down"] = ""
    if direction == 'down':
      connected_location.connections["up"] = self
      connected_location.travel_descriptions["up"] = ""
    if direction == 'in':
      connected_location.connections["out"] = self
      connected_location.travel_descriptions["out"] = ""
    if direction == 'out':
      connected_location.connections["in"] = self
      connected_location.travel_descriptions["in"] = ""


  def add_item(self, name, item):
    """Put an item in this location."""
    self.items[name] = item

  def remove_item(self, item):
    """Remove an item from this location (for instance, if the player picks it
       up and puts it in their inventory)."""
    self.items.pop(item.name)


  def is_blocked(self, direction, game):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
        return False
    (block_description, preconditions) = self.blocks[direction]
    if check_preconditions(preconditions, game):
      # All the preconditions have been met.  You may pass.
      return False
    else: 
      # There are still obstalces to overcome or puzzles to solve.
      return True

  def get_block_description(self, direction):
    """Check to if there is an obstacle in this direction."""
    if not direction in self.blocks:
      return ""
    else:
      (block_description, preconditions) = self.blocks[direction]
      return block_description

  def add_block(self, blocked_direction, block_description, preconditions):
    """Create an obstacle that prevents a player from moving in the blocked 
       location until the preconditions are all met."""
    self.blocks[blocked_direction] = (block_description, preconditions)

## Checking Preconditions 
In text adventure games it's common to block a player's progress by creating blocks that prevent them from moving to a location.  For instance, a drawbridge might have a troll that you need to get rig of before you can cross into the castle, or a locked door might prevent you from entering a building until you have a key.  

This is a function that you can modify to include other preconditions.

In [0]:
def check_preconditions(preconditions, game, print_failure_reasons=True):
  """Checks whether the player has met all of the specified preconditions"""
  all_conditions_met = True
  for check in preconditions: 
    if check == "inventory_contains":
      item = preconditions[check]
      if not game.is_in_inventory(item):
        all_conditions_met = False
        if print_failure_reasons:
          print("You don't have the %s" % item.name)
    if check == "in_location":
      location = preconditions[check]
      if not game.curr_location == location:
        all_conditions_met = False
        if print_failure_reasons:
          print("You aren't in the correct location")
    if check == "location_has_item":
      item = preconditions[check]
      if not item.name in game.curr_location.items:
        all_conditions_met = False
        if print_failure_reasons:
          print("The %s isn't in this location" % item.name)
    # todo - add other types of preconditions
    if check == "location_does_not_have_item":
      item = preconditions[check]
      if item.name in game.curr_location.items:
        all_conditions_met = False
        if print_failure_reasons:
          print("The %s is still in this location" % item.name)



    
  return all_conditions_met

## Items
Items are objects that a player can get, or scenery that a player can examine. We could also implement people as items.  

In [0]:
class Item:
  """Items are objects that a player can get, or scenery that a player can
     examine."""
  def __init__(self,
               name,
               description,
               examine_text="",
               take_text="",
               start_at=None,
               gettable=True,
               end_game=False):
    # The name of the object
    self.name = name
    # The default description of the object.
    self.description = description
    # The detailed description of the player examines the object.
    self.examine_text = examine_text
    # Text that displays when player takes an object.
    self.take_text = take_text if take_text else ("You take the %s." % self.name)
    # Indicates whether a player can get the object and put it in their inventory.
    self.gettable = gettable
    # True if entering this location should end the game.
    self.end_game = end_game
    # The location in the Game where the object starts.
    if start_at:
      start_at.add_item(name, self)
    self.commands = {}


  def get_commands(self):
    """Returns a list of special commands associated with this object"""
    return self.commands.keys()

  def add_action(self, command_text, function, arguments, preconditions={}):
    """Add a special action associated with this item"""
    self.commands[command_text] = (function, arguments, preconditions)

  def do_action(self, command_text, game):
    """Perform a special action associated with this item"""
    end_game = False  # Switches to True if this action ends the game.
    if command_text in self.commands:
      function, arguments, preconditions = self.commands[command_text]
      if check_preconditions(preconditions, game):
        end_game = function(game, arguments)
    else:
      print("Cannot perform the action %s" % command_text)
    return end_game

## The Parser
The parser is the module that handles the natural language understanding in the game.  The players enter commands in text, and the parser interprets them and performs the actions that the player intends.  This is the module with the most potential for improvement using modern natural language processing.  The implementation that I have given below only uses simple keyword matching.

In [0]:
class Parser:
  """The Parser is the class that handles the player's input.  The player 
     writes commands, and the parser performs natural language understanding
     in order to interpret what the player intended, and how that intent
     is reflected in the simulated world. 
  """
  def __init__(self, game):
    # A list of all of the commands that the player has issued.
    self.command_history = []
    # A pointer to the game.
    self.game = game

  def get_player_intent(self,command):
    command = command.lower()
    if "," in command:
      # Let the player type in a comma separted sequence of commands
      return "sequence"
    elif self.get_direction(command):
      # Check for the direction intent
      return "direction"
    elif command.lower() == "look" or command.lower() == "l":
      # when the user issues a "look" command, re-describe what they see
      return "redescribe"
    elif "examine " in command or command.lower().startswith("x "):
      return "examine"
    elif  "take " in command or "get " in command:
      return "take"
    elif "drop " in command:
      return "drop"
    elif "inventory" in command or command.lower() == "i":
      return "inventory"
    else: 
      for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return "special"

  def parse_command(self, command):
    # add this command to the history
    self.command_history.append(command)

    # By default, none of the intents end the game. The following are ways this
    # flag can be changed to True.
    # * Going to a certain place.
    # * Entering a certain special command
    # * Picking up a certain object.

    end_game = False

    # Intents are functions that can be executed
    intent = self.get_player_intent(command)
    if intent == "direction":
      end_game = self.go_in_direction(command)
    elif intent == "redescribe":
      self.game.describe()
    elif intent == "examine":
      self.examine(command)
    elif intent == "take":
      end_game = self.take(command)
    elif intent == "drop":
      self.drop(command)
    elif intent == "inventory":
      self.check_inventory(command)
    elif intent == "special":
      end_game = self.run_special_command(command)
    elif intent == "sequence":
      end_game = self.execute_sequence(command)
    else:
      print("I'm not sure what you want to do.")
    return end_game

  ### Intent Functions ###

  def go_in_direction(self, command):
    """ The user wants to in some direction """
    direction = self.get_direction(command)

    if direction:
      if direction in self.game.curr_location.connections:
        if self.game.curr_location.is_blocked(direction, self.game):
          # check to see whether that direction is blocked.
          print(self.game.curr_location.get_block_description(direction))
        else:
          # if it's not blocked, then move there 
          self.game.curr_location = self.game.curr_location.connections[direction]

          # If moving to this location ends the game, only describe the location
          # and not the available items or actions.
          if self.game.curr_location.end_game:
            self.game.describe_current_location()
          else:
            self.game.describe()
      else:
        print("You can't go %s from here." % direction.capitalize())
    return self.game.curr_location.end_game

  def check_inventory(self,command):
    """ The player wants to check their inventory"""
    if len(self.game.inventory) == 0:
      print("You don't have anything.")
    else:
      descriptions = []
      for item_name in self.game.inventory:
        item = self.game.inventory[item_name]
        descriptions.append(item.description)
      print("You have: ", end = '')
      print(*descriptions, sep = ", ",)
  

  def examine(self, command):
    """ The player wants to examine something """
    command = command.lower()
    matched_item = False
    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
        break
    # check whether any of the items in the inventory match the command
    for item_name in self.game.inventory:
      if item_name in command:
        item = self.game.inventory[item_name]
        if item.examine_text:
          print(item.examine_text)
          matched_item = True
    # fail
    if not matched_item:
      print("You don't see anything special.")


  def take(self, command):
    """ The player wants to put something in their inventory """
    command = command.lower()
    matched_item = False

    # This gets set to True if posession of this object ends the game.
    end_game = False

    # check whether any of the items at this location match the command
    for item_name in self.game.curr_location.items:
      if item_name in command:
        item = self.game.curr_location.items[item_name]
        if item.gettable:
          self.game.add_to_inventory(item)
          self.game.curr_location.remove_item(item)
          print(item.take_text)
          end_game = item.end_game
        else:
          print("You cannot take the %s." % item_name)
        matched_item = True
        break
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          print("You already have the %s." % item_name)
          matched_item = True
    # fail
    if not matched_item:
      print("You can't find it.")

    return end_game

  def drop(self, command):
    """ The player wants to remove something from their inventory """
    command = command.lower()
    matched_item = False
    # check whether any of the items in the inventory match the command
    if not matched_item:
      for item_name in self.game.inventory:
        if item_name in command:
          matched_item = True
          item = self.game.inventory[item_name]
          self.game.curr_location.add_item(item_name, item)
          self.game.inventory.pop(item_name)
          print("You drop the %s." % item_name)
          break
    # fail
    if not matched_item:
      print("You don't have that.")


  def run_special_command(self, command):
    """Run a special command associated with one of the items in this location
       or in the player's inventory"""
    for item in self.game.get_items_in_scope():
        special_commands = item.get_commands()
        for special_command in special_commands:
          if command == special_command.lower():
            return item.do_action(special_command, self.game)

  def execute_sequence(self, command):
    for cmd in command.split(","):
      cmd = cmd.strip()
      self.parse_command(cmd)

  def get_direction(self, command):
    command = command.lower()
    if command == "n" or "north" in command:
      return "north" 
    if command == "s" or "south" in command:
      return "south"
    if command == "e" or "east" in command: 
      return "east"
    if command == "w" or "west" in command:
      return "west"
    if command == "up":
      return "up"
    if command == "down":
      return "down"
    if command.startswith("go out"):
      return "out"
    if command.startswith("go in"):
      return "in"
    for exit in self.game.curr_location.connections.keys():
      if command == exit.lower() or command == "go " + exit.lower():
        return exit
    return None

## Special functions
Many times we want to add special behavior to items in the game.  For instance, we might want to be able to _pick a rose_ from a _rosebush_, or the _eat_ a _fish_.  In this implementation we do this in a pretty generic way by allowing the game developer to call ```Item.add_action(cmd,function,argment,preconditions)``` where ```function``` is any Python function. Some example of functions are defined below.

These functions should return True if the game is ended by the action, False otherwise.

In [9]:
import nltk
nltk.download('punkt')
from textblob import TextBlob

def add_item_to_inventory(game, *args):
  """ Add a newly created Item and add it to your inventory."""
  (item, action_description, already_done_description) = args[0]
  if(not game.is_in_inventory(item)):
    print(action_description)
    game.add_to_inventory(item)
  else:
    print(already_done_description)
  return False

def describe_something(game, *args):
  """Describe some aspect of the Item"""
  (description) = args[0]
  print(description)
  return False

def say_happy_things(game, *args):
  print("Say something positive to start the day right!")
  (location) = args[0]
  text = input(">")
  blob = TextBlob(text)

  polarity_score = 0

  for sentence in blob.sentences:
    polarity_score += sentence.sentiment.polarity

  if polarity_score < 0:
    print("Not with that attitude. YOU LOSE!")
    return True
  if polarity_score < 0.3:
    print("Good enough I guess...")
  else:
    print("That's the spirit!")
  
  game.curr_location = location
  game.describe_current_location()
  game.describe_items()

  return False


def destroy_item(game, *args):
  """Removes an Item from the game by setting its location is set to None."""
  (item, action_description) = args[0]
  if game.is_in_inventory(item):
    game.inventory.pop(item.name)
    print(action_description)
  elif item.name in game.curr_location.items:
    game.curr_location.remove_item(item)
    print(action_description)
  return False

def create_item(game, *args):
  (item, action_description) = args[0]
  game.curr_location.add_item(item.name, item)
  print(action_description)
  return False

def destroy_items(game, *args):
  for item in args[0]:
    destroy_item(game, item)
  return False

def perform_multiple_actions(game, *args):
  for action_pair in args[0]:
    (action, action_args) = action_pair
    action(game, action_args)
  return False

def create_item_in_loc(game, *args):
  (item, location) = args[0]
  current_location = game.curr_location
  game.curr_location = location
  game.curr_location.add_item(item.name, item)
  game.curr_location = current_location
  return False

def destroy_item_in_loc(game, *args):
  (item, location) = args[0]
  current_location = game.curr_location
  game.curr_location = location
  destroy_item(game, (item, ""))
  game.curr_location = current_location
  return False


def end_game(game, *args):
  """Ends the game."""
  end_message = args[0]
  print(end_message)
  return True

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Game Data

Here's where you can define the locations and items in your game.  To get you started, I defined a super-simple fishing game, which contains the first 3 locations of __Action Castle__ by Jared A. Sorensen, which is part of the awesome book [Parsley](http://www.memento-mori.com/parsely-products/parsely-pdf).  

You can play through the whole game with the following commands:
1. take pole
2. go out
3. south 
4. catch fish with pole
5. eat fish

In [0]:
def build_game():
  # Locations
  #example: cottage = Location("Cottage", "You are standing in a small cottage.")
  mind_palace = Location("Mind Palace", "You open your little rat eyes to the world. The darkness of your sewer home is a little too bright for the moment. \n It takes a while before you can open your eyes fully. Your cozy blanket keeps you shielded from the fumes of sewer air. \n It's a good day. You are still half asleep and up here in your head. Take your time to come out. \n It can be hard to take on the day. You are RATSO, and this is 'The Tale of DespeRatso.'")                         
  bedroom = Location("Bedroom", "This is your room. You have a soft bed and a cute rug in a den you built yourself.")
  kitchen = Location("Kitchen", "This is your kitchen. Maybe there is something in the pantry to make a little snack for the day.")
  pantry = Location("Pantry", "Your pantry! Full of snacks and ingredients!")
  hookah_den = Location("Hookah Den", "This is your hookah den. Even rats like it!")
  first_ledge = Location("First Ledge", "You hang from the first ledge on the climb out of the sewers.")
  second_ledge = Location("Second Ledge", "You make it up to the second ledge. It's very high.")
  third_ledge = Location("Third Ledge", "Ugh another one down! Almost there!")
  fourth_ledge = Location("Fourth Ledge", "The last ledge! You can see the sunlight coming through the sewer grate above.")
  window_ledge = Location("Window", "A window to another rat's home. It's dirty but you could probably see inside if you tried.")
  maze_1 = Location("Maze 1", "All the ledges in this part look the same...")
  maze_2 = Location("Maze 2", "All the ledges in this part look the same...")
  maze_3 = Location("Maze 3", "All the ledges in this part look the same...")
  surface = Location("Surface", "The sun beats down on you. It is so bright you have to close your eyes for a moment while they adjust.")


  # Connections
  #example: cottage.add_connection("out", garden_path)
  #mind_palace.add_connection("out", bedroom)
  bedroom.add_connection("west", kitchen)
  bedroom.add_connection("east", hookah_den)
  bedroom.add_connection("up", first_ledge)
  kitchen.add_connection("in", pantry)
  first_ledge.add_connection("up", second_ledge)
  second_ledge.add_connection("west", maze_1)
  second_ledge.add_connection("east", third_ledge)
  third_ledge.add_connection("west", window_ledge)
  third_ledge.add_connection("east", fourth_ledge)
  fourth_ledge.add_connection("up", surface)
  maze_1.add_connection("north", maze_2)
  maze_1.add_connection("south", maze_3)
  maze_2.add_connection("north", maze_3)



  # Items that you can pick up
  #example: fishing_pole = Item("pole", "a fishing pole", "A SIMPLE FISHING POLE.", start_at=cottage)
  slippers = Item("slippers", "your slippers", "THEY HAVE LITTLE RAT FACES ON THEM.", start_at=bedroom)
  apron = Item("apron", "your apron", "IT SAYS 'KISS THE CHEF' (VERY POLITELY).", start_at=kitchen)
  cheese = Item("cheese", "some cheese", "MICE LIKE THIS!", start_at=pantry)
  strawberry = Item("strawberry", "a single strawberry", "IT'S THE SIZE OF YOUR HEAD!", start_at=pantry)
  peanut_butter = Item("peanut butter", "a clump of peanut butter", "YOUR TONGUE FEELS ITCHY THINKING ABOUT IT.", start_at=pantry)
  snack = Item("snack", "a lovely snack", "JUST LIKE UNCLE REMY WOULD MAKE!", start_at=None)
  cloudy_mind = Item("cloudy mind", "cloudy mind", "YOU FEEL MUCH CALMER.", start_at=None)
  hawk_repellant = Item("hawk repellant", "hawk repellant", "IT SAYS 'HAWK REPELLANT', BUT WHO'S TO SAY?", start_at=maze_3)

  # Sceneary (not things that you can pick up)
  #example: pond = Item("pond", "a small fishing pond", "THERE ARE FISH IN THE POND.", start_at=fishing_pond, gettable=False)
  happy_thoughts = Item("", "Nothing... press enter to get started", "", start_at=mind_palace, gettable=False)
  hookah = Item("hookah", "a hookah", "LOTS OF MEMORIES WITH THIS...", start_at=hookah_den, gettable=False)
  pot = Item("pot", "a pot", "YOU COOK IN THIS.", start_at=kitchen, gettable=False)
  cheese_pot = Item("pot", "a pot", "YOU COOK IN THIS.", start_at=None, gettable=False)
  strawberry_pot = Item("pot", "a pot", "YOU COOK IN THIS.", start_at=None, gettable=False)
  bed = Item("bed", "your bed", "YOU CAN GO BACK TO SLEEP", start_at=bedroom, gettable=False)
  window = Item("window", "a window", "A RAT FAMILY LIVES HERE.", start_at=window_ledge, gettable=False)
  sun = Item("sun in your eyes", "the sun is in your eyes", "IT HURTS TO LOOK AT.", start_at=surface, gettable=False)

  #Blocks
  #example: draw_bridge.add_block("east", "The troll blocks your way. Spittle falls from its mouth.", preconditions={"location_does_not_have_item": troll})
  bedroom.add_block("east", "You should probably wear some slippers before you go about your day.", preconditions={"inventory_contains": slippers})
  bedroom.add_block("west", "You should probably wear some slippers before you go about your day.", preconditions={"inventory_contains": slippers})
  bedroom.add_block("up", "You should probably wear some slippers before you go about your day.", preconditions={"inventory_contains": slippers})
  first_ledge.add_block("up", "You are suddenly overwhelmed by your fear of heights and begin shaking. Maybe it's best not to climb any higher until you calm down.", preconditions={"inventory_contains": cloudy_mind})
  fourth_ledge.add_block("up", "Your stomach growls. Maybe you should bring a snack before you venture up on the surface for the day.", preconditions={"inventory_contains": snack})


  # Add special functions to your items
  #examples: rosebush.add_action("pick rose",  add_item_to_inventory, (rose, "You pick the lone rose from the rosebush.", "You already picked the rose."))
  #rose.add_action("smell rose",  describe_something, ("It smells sweet."))
  #long_fall.add_action("jump", end_game, ("You jump off the tree and break your legs. You do not die, until you suddenly do."))
  #troll.add_action("give fish to troll", destroy_items, 
  #                 ((fish, "The troll seems satisfied after devouring the fish."), (troll, "He stomps away happy about making a friend today. He cannot wait to tell his mother, who is also a troll.")), preconditions={"inventory_contains":fish})
  #guard.add_action("hit guard with branch", perform_multiple_actions, 
  #    ([(destroy_item, (branch,"You swing your branch against the guard. It shatters to pieces.")),
  #    (destroy_item, (guard,"The guard slumps over, unconscious.")),
  #    (create_item, (unconscious_guard,"The guard's unconscious body lies on the ground.")),
  #    (create_item, (key,"His key falls from his hand.")),
  #    ]), preconditions={"inventory_contains":branch , "location_has_item": guard})
  #princess.add_action("marry princess", perform_multiple_actions,
  #                    ([(describe_something, ("You propose to the princess. \n 'My father’s crown! You have put his soul at rest and may now succeed him!' \n You and the princess are now married. She places the crown on your head.")),
  #                    (destroy_item, (crown, "")), 
  #                    (destroy_item, (princess, "")), 
  #                    (create_item, (queen, "")),
  #                    (add_item_to_inventory, (wedding_ring, "", "")),
  #                    (create_item_in_loc, (new_guard, court_yard)),
  #                    (create_item_in_loc, (revelers, feasting_hall)),
  #                    (create_item_in_loc, (revelers, throne_room)),
  #                    (destroy_item_in_loc, (unconscious_guard, court_yard))
  #                    ]), preconditions={"has_crown": crown})

  pot.add_action("add cheese to pot", perform_multiple_actions, 
                 ([(destroy_item, (pot, "You add cheese to the pot!")), 
                   (create_item, (cheese_pot, "The cheese smells so good!")),
                  (destroy_item, (cheese, ""))]), preconditions={"inventory_contains": cheese, "inventory_contains": apron})
  pot.add_action("add strawberry to pot", perform_multiple_actions, 
                 ([(destroy_item, (pot, "You add the strawberry to the pot!")), 
                   (create_item, (strawberry_pot, "The strawberry is cooking I guess!")),
                  (destroy_item, (strawberry, ""))]), preconditions={"inventory_contains": strawberry, "inventory_contains": apron} )
  pot.add_action("add peanut butter to pot", end_game, ("The kitchen fills with fumes of peanut butter and sewage. \n You suddenly remember how in Rat Chemistry class that peanut butter and sewer air catalyzes into- YOU ARE DEAD. PAINLESS(LESS)LY."), preconditions={"inventory_contains": peanut_butter, "inventory_contains": apron} )
  cheese_pot.add_action("add peanut butter to pot", end_game, ("The kitchen fills with fumes of peanut butter and sewage. \n You suddenly remember how in Rat Chemistry class that peanut butter and sewer air catalyzes into- YOU ARE DEAD. PAINLESS(LESS)LY."), preconditions={"inventory_contains": peanut_butter, "inventory_contains": apron} )
  strawberry_pot.add_action("add peanut butter to pot", end_game, ("The kitchen fills with fumes of peanut butter and sewage. \n You suddenly remember how in Rat Chemistry class that peanut butter and sewer air catalyzes into- YOU ARE DEAD. PAINLESS(LESS)LY."), preconditions={"inventory_contains": peanut_butter, "inventory_contains": apron} )
  strawberry_pot.add_action("add cheese to pot", perform_multiple_actions, 
                 ([(destroy_item, (strawberry_pot, "You add cheese to the pot!")), 
                   (create_item, (pot, "")),
                   (create_item, (snack, "A lovely snack!")),
                  (destroy_item, (cheese, ""))]), preconditions={"inventory_contains": cheese, "inventory_contains": apron})
  cheese_pot.add_action("add strawberry to pot", perform_multiple_actions, 
                 ([(destroy_item, (cheese_pot, "You add the strawberry to the pot!")), 
                   (create_item, (pot, "")),
                   (create_item, (snack, "A lovely snack!")),
                  (destroy_item, (strawberry, ""))]), preconditions={"inventory_contains": strawberry, "inventory_contains": apron})
  cheese.add_action("eat cheese", describe_something, ("Uncooked? Absurd!"))
  strawberry.add_action("eat strawberry", describe_something, ("Uncooked? Absurd!"))
  peanut_butter.add_action("eat peanut butter", describe_something, ("Uncooked? Absurd!"))

  hookah.add_action("use hookah", add_item_to_inventory, (cloudy_mind, "You use the hookah. Your head becomes cloudy but you are no longer stressed.", "You probably should be responsible."))

  window.add_action("look in window", end_game, ("Inside you can see... OMG YOU MUST GET HELP. CALL THE RAT POLICE! 9-1-1! In your panic you accidentally take a step backwards off the ledge and fall into darkness. YOU LOSE!"))
  snack.add_action("eat snack", end_game, ("You never quite learned portion control when cooking. You are too full to move. You take a nap instead and hibernate for the rest of your rat-life span. You are dead. YOU LOSE!"))
  
  sun.add_action("open eyes", end_game, ("You open your eyes to notice a large hawk standing over you. You scream in fear. Fortunately, the hawk is friendly. Unfortunately, you now have to live with the knowledge that you harbor unaddressed internalized racism. The day is ruined. YOU LOSE!"))
  sun.add_action("use hawk repellant", end_game, ("While your eyes adjust you spray some hawk repellant. Opening your eyes you see a bright and shining world. Time for the day to begin! YOU WIN!"), preconditions={"inventory_contains": hawk_repellant, "in_location": surface})

  happy_thoughts.add_action("", say_happy_things, (bedroom))

  game = Game(mind_palace)
  return game


# Play the game
This small snippet of code is what you need to run the game.  Behold! The magestic prompt! 

In [8]:
def game_loop():
  game = build_game()
  parser = Parser(game)
  game.describe()

  command = ""
  while not (command.lower() == "exit" or command.lower == "q"):
    command = input(">")
    end_game = parser.parse_command(command)
    if end_game:
      return

game_loop()
print('THE GAME HAS ENDED.')

You open your little rat eyes to the world. The darkness of your sewer home is a little too bright for the moment. 
 It takes a while before you can open your eyes fully. Your cozy blanket keeps you shielded from the fumes of sewer air. 
 It's a good day. You are still half asleep and up here in your head. Take your time to come out. 
 It can be hard to take on the day. You are RATSO, and this is 'The Tale of DespeRatso.'
You see: 
Nothing... press enter to get started
	 
>
Say something positive to start the day right!
>fuck u


NameError: ignored

# Visualize your game
The code below allows you to create a directed graph that shows the locations in your game and how they are connected.  You can also save a PDF of your graph to your Google Drive with the `save_to_drive` method.  The output file will be called `game-visualization.pdf`.

In [0]:
#!pip install graphviz
from graphviz import Digraph
from IPython.display import Image
import queue

def DFS(game, graph):
  """Do a depth-first-search traversal of the locations in the game
     starting at the start location, and create a GraphViz graph 
     to vizualize the connections between the locations, and the items
     that are located at each location."""
  start_location = game.curr_location
  frontier = queue.Queue()
  frontier.put(start_location)
  visited = {}
  visited[start_location.name] = True

  while not frontier.empty():
    current_location = frontier.get()
    game.curr_location = current_location
    name = current_location.name
    description = current_location.description
    items = current_location.items
    items_html = describe_items(current_location)
    html = "<<b>%s</b><br />%s<br />%s>" % (name, description, items_html)
    # Create a new node in the graph for this location
    graph.node(name, label=html)  

    connections = current_location.connections
    for direction in connections.keys():
      next_location = connections[direction]
      if not current_location.is_blocked(direction, game):
        # Create an edge between the current location and its successor
        graph.edge(name, next_location.name, label=direction.capitalize())
      else:
        # Create a dotted edge for connected locations that are blocked
        block_description = "%s\n%s" % (direction.capitalize(), current_location.get_block_description(direction))
        graph.edge(name, next_location.name, label=block_description, style="dotted")
      if not next_location.name in visited:
        visited[next_location.name] = True
        frontier.put(next_location)

def describe_items(location, print_commands=True):
    """Describe what objects are in the current location."""
    items_html = ""
    if len(location.items.keys()) > 0:
      items_html = "You see: "
    for item_name in location.items:
      item = location.items[item_name]
      items_html += item.description
      if print_commands:
        special_commands = item.get_commands()
        for cmd in special_commands:
          items_html += "<br/><i>%s</i>" % cmd
    return items_html

def save_to_drive(graph):
  from google.colab import drive
  drive.mount('/content/drive/')
  graph.render('/content/drive/My Drive/game-visualization', view=True)  

graph = Digraph(node_attr={'color': 'lightblue2', 'style': 'filled'})
game = build_game()
DFS(game, graph)
save_to_drive(graph)
graph
